In [12]:
%pylab inline
import numpy as np
import pandas as pd
data = []
datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.01.24.csv'
data = pd.read_csv(datapath)
data_CNF14 = data[data.Series == 'CNF14']
data = data_CNF14
data.head()

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNF14,1,2014-01-24D04:21:49.419665000,C791DA0F799A5E9A,1,710000,2,,A,677000,19
1,CNF14,2,2014-01-24D04:21:49.419665000,E1C9F25394A679A6,2,719500,1,,A,677000,19
2,CNF14,3,2014-01-24D04:21:49.419665000,C02B6F481F080128,1,684000,100,,A,677000,19
3,CNF14,4,2014-01-24D04:21:49.419665000,C55C20A9615D40FD,2,692500,1,,A,677000,19
4,CNF14,5,2014-01-24D04:21:49.419665000,E5D428BDEDEDCCCD,1,678500,20,,A,677000,19


In [14]:
data_CNF = data[data.Series == data.Series[0]]
data = data_CNF
len(data)

143284

In [28]:
import time 
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

order_book_bid.append(0)
order_book_ask.append(0)
#a = []
#b = []

for i in range(len(y),3200,1):
    print '------------------------------------------------------------------------------'
    print i
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':

        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                print order_book_ask[temp_ask]
                #print temp_ask
                if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    print 'Best Price is True'
                    continue
                else:
                    print 'Best Price is False'
                    break
                    
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_ask[temp_ask]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print 'position = %d'%(position_)
                    #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                    #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                        
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue                            
                            
                            
                            
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error1111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error2222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error3333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    print 'Best Price is True'
                    continue
                else:
                    print 'Best Price is False'
                    break
                    
                order_book_ask.append(order_book_ask[temp_ask])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error11111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error22222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error33333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue 
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue                          

    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
            print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                #print position_
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error222222 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error333333 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue
                
            elif data.TimeStamp[i] != x2[temp_bid]:
                ###b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    print 'Best Price is True'
                    continue
                else:
                    print 'Best Price is False'
                    break
                    
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    print data['BestPrice'][i]
                    print order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0]
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1111111 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                            
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            continue
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print temp_bid
                #print order_book_bid[temp_bid]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error19 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error29 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error39 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x2[temp_bid]:
                if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    print 'Best Price is True'
                    continue
                else:
                    print 'Best Price is False'
                    break                
                ###b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error110 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
    
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error220 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error330 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue                    
                                    
end = time.time()            
print "Total time = %f"%(end - start)    

------------------------------------------------------------------------------
12
False
Best Price is False
Total time = 0.145939


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [26]:
def order_book_(data,data_number):
    def insert(order_book_data,data_to_insert,ob_position):
        top = order_book_data[0:ob_position]
        bottom = order_book_data[ob_position:]
        return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

    order_book_bid = []
    order_book_ask = []
    timestamp = data.TimeStamp[0]
    temp_ask = 0
    temp_bid = 0
    bid = []
    ask = []
    timestamp_ = []
    x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
    x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
    y = data[data['TimeStamp'].str.contains(timestamp)]
    bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
    ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
    a = bid[0].sort(['Price'],ascending = [False])
    b = ask[0].sort(['Price'],ascending = [True])    
    order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
    order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

    order_book_bid.append(0)
    order_book_ask.append(0)
    #a = []
    #b = []
    
    for i in range(len(y),data_number,1):
        print '------------------------------------------------------------------------------'
        print i
        #print data.BidOrAsk[i] == 'A'
        if data.BidOrAsk[i] == 'A':

            if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
                if data.TimeStamp[i] == x1[temp_ask]:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'   
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                            
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True' 
                
                elif data.TimeStamp[i] != x1[temp_ask]:
                    ###a.append(x1[temp_ask])
                    #print order_book_ask[temp_ask]
                    #print temp_ask
                    #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    #    print 'Best Price is True'
                    #    continue
                    #else:
                    #    print 'Best Price is False'
                    #    break

                    if temp_ask == 0:
                        temp_ask = temp_ask + 1
                        order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                        #print order_book_ask[temp_ask]
                        order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                        #print order_book_ask[temp_ask]
                        best_price = data[i:(i+1)]['BestPrice']
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                        #print 'position = %d'%(position_)
                        #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                        #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                        #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                        #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                        if position_ == 0 and len(order_book_ask[0]) > 1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error11 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                continue
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:

                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error22 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'  
                                continue
                        elif position_ == len(order_book_ask[temp_ask])-1:

                            if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error33 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                continue
                            else:
                                print 'position and Best Price is True'
                                continue
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            continue

                        #print order_book_ask[temp_ask]                        
                    else:
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                        order_book_ask.append(order_book_ask[temp_ask])
                        temp_ask = temp_ask + 1

                        order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)


                        if position_ == 0:
                            if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                print 'Some error111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                continue
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:

                            if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                print 'Some error222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'  
                                continue
                        elif position_ == len(order_book_ask[temp_ask])-1:

                            if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                        
                                continue
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            continue                            



            elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

                if data.TimeStamp[i] == x1[temp_ask]: 
                    #print temp_ask
                    #print order_book_ask[temp_ask]
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                    price_ = data[i:i+1]['Price'].iloc[0]

                    if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                        order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                    else:
                        order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    #print order_book_ask[temp_ask]

                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error1111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error2222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:

                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error3333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            continue                        
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue

                elif data.TimeStamp[i] != x1[temp_ask]:
                    ###a.append(x1[temp_ask])
                    #print temp_ask
                    #print order_book_ask[temp_ask]
                    #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    #    print 'Best Price is True'
                    #    continue
                    #else:
                    #    print 'Best Price is False'
                    #    break

                    order_book_ask.append(order_book_ask[temp_ask])
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                    price_ = data[i:i+1]['Price'].iloc[0]
                    temp_ask = temp_ask + 1
                    #print 'temp_ask2 = %d'%(temp_ask)
                    if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                        order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                    else:
                        order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    #print order_book_ask[temp_ask]

                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error11111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error22222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:

                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error33333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            continue                        
                        else:
                            print 'position and Best Price is True'                        
                            continue 
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue                          

        elif data.BidOrAsk[i] == 'B':
            #print i
            #print int(data[['QuantityDifference']][i:i+1].values) > 0

            if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
                print data.TimeStamp[i] == x2[temp_bid]

                if data.TimeStamp[i] == x2[temp_bid]:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_

                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue

                    elif 0 < position_< len(order_book_bid[temp_bid])-1:

                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error222222 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'   
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:

                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error333333 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue

                elif data.TimeStamp[i] != x2[temp_bid]:
                    ###b.append(x2[temp_bid])
                    #print 'temp bid = %d'%(temp_bid)
                    #print order_book_bid[temp_bid]
                    #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    #    print 'Best Price is True'
                    #    continue
                    #else:
                    #    print 'Best Price is False'
                    #    break

                    if temp_bid == 0:
                        #print temp_bid
                        #print order_book_bid[temp_bid]
                        temp_bid = temp_bid + 1
                        order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                        #print order_book_bid[temp_bid]
                        order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                        #print order_book_bid[temp_bid]
                        best_price = data[i:(i+1)]['BestPrice']
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                        #print position_
                        #print len(order_book_bid[temp_bid]) - 1

                        #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                        #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                        #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                        #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                        #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                        #print temp_bid
                        #print order_book_bid[temp_bid]
                        print data['BestPrice'][i]
                        print order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0]
                        if position_ == 0  and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error1111111 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                continue

                        elif 0 < position_< len(order_book_bid[temp_bid])-1:

                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error2222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'  
                                continue
                        elif position_ == len(order_book_bid[temp_bid])-1:

                            if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error3333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                continue
                            else:
                                print 'position and Best Price is True'
                                continue
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                                print 'position and Best Price is True'
                                continue
                    else:
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                        order_book_bid.append(order_book_bid[temp_bid])
                        temp_bid = temp_bid + 1
                        order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                        #print position_    
                        #print len(order_book_bid[temp_bid])-1      
                        if position_ == 0 and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error11111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                continue
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:

                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error22222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'  
                                continue
                        elif position_ == len(order_book_bid[temp_bid])-1:

                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                print 'Some error33333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                        
                                continue
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            continue

            elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

                if data.TimeStamp[i] == x2[temp_bid]: 
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                    price_ = data[i:i+1]['Price'].iloc[0]

                    if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                        order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                    else:
                        order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    #print order_book_bid[temp_bid]

                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error19 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue

                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error29 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:

                        if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error39 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            continue
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue

                elif data.TimeStamp[i] != x2[temp_bid]:
                    #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                    #    print 'Best Price is True'
                    #    continue
                    #else:
                    #    print 'Best Price is False'
                    #    break                
                    ###b.append(x2[temp_bid])
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    order_book_bid.append(order_book_bid[temp_bid])
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                    price_ = data[i:i+1]['Price'].iloc[0]
                    temp_bid = temp_bid + 1
                    #print position_
                    #print len(order_book_ask[temp_ask])-1
                    #print 'temp_bid2 = %d'%(temp_bid)

                    if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                        order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                    else:
                        order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    #print order_book_bid[temp_bid]

                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error110 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue

                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error220 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:

                        if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error330 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            continue
                        else:
                            print 'position and Best Price is True'                        
                            continue    
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue                    
    return 
    #return len(data)

In [27]:
order_book_(data,20)

------------------------------------------------------------------------------
12
False
674500
672500
position and Best Price is True
------------------------------------------------------------------------------
13
True
position and Best Price is True
------------------------------------------------------------------------------
14
True
position and Best Price is True
------------------------------------------------------------------------------
15
True
position and Best Price is True
------------------------------------------------------------------------------
16
position and Best Price is True
------------------------------------------------------------------------------
17
position and Best Price is True
------------------------------------------------------------------------------
18
position and Best Price is True
------------------------------------------------------------------------------
19
position and Best Price is True


/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


143284

In [21]:
data.head(30)

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNF14,1,2014-01-24D04:21:49.419665000,C791DA0F799A5E9A,1,710000,2,,A,677000,19
1,CNF14,2,2014-01-24D04:21:49.419665000,E1C9F25394A679A6,2,719500,1,,A,677000,19
2,CNF14,3,2014-01-24D04:21:49.419665000,C02B6F481F080128,1,684000,100,,A,677000,19
3,CNF14,4,2014-01-24D04:21:49.419665000,C55C20A9615D40FD,2,692500,1,,A,677000,19
4,CNF14,5,2014-01-24D04:21:49.419665000,E5D428BDEDEDCCCD,1,678500,20,,A,677000,19
5,CNF14,6,2014-01-24D04:21:49.419665000,C1442DE9FABCDEFC,2,679500,20,,A,677000,19
6,CNF14,7,2014-01-24D04:21:49.419665000,CD0701B13A4F1E8F,1,674000,10,,B,674000,20
7,CNF14,8,2014-01-24D04:21:49.419665000,CB1CA0F570654F45,2,672500,20,,B,674000,20
8,CNF14,9,2014-01-24D04:21:49.419665000,E94584AD9F7F833F,1,677000,19,,A,677000,19
9,CNF14,10,2014-01-24D04:21:49.419665000,9672009CE4FAC9FA,5,689500,2,,A,677000,19


In [42]:
a = 1
b = 2
if a > b:
    print a
    pass
elif a < b:
    print a
    pass


1


In [29]:
order_book_ask[0]#[(order_book_ask[0] == 'A')]

,Price,OrderNumber,QuantityDifference
0,677000,E94584AD9F7F833F,19
1,677500,CFCC80F56C3C4FFC,20
2,678500,E5D428BDEDEDCCCD,20
3,679500,C1442DE9FABCDEFC,20
4,684000,C02B6F481F080128,100
5,689500,9672009CE4FAC9FA,2
6,692500,C55C20A9615D40FD,1
7,710000,C791DA0F799A5E9A,2
8,719500,E1C9F25394A679A6,1


In [64]:
import time 
data_ask = data[(data.BidOrAsk == 'A')].reindex()
data_bid = data[(data.BidOrAsk == 'B')].reindex()

def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

start  =  time.time()

order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))

order_book_bid.append(0)
order_book_ask.append(0)
#a = []
#b = []
data_ask_number = len(order_book_ask[0])
data_bid_number = len(order_book_bid[0])

for i in range(len(y),3200,1):
    print '------------------------------------------------------------------------------'
    print i
    print data[['TimeStamp']][i:i+1]
    print data[['Price','OrderNumber','QuantityDifference','BidOrAsk','BestPrice']][i:i+1]
    data_ask_number = data_ask_number + 1
    print 'data_ask_number = %d'%(data_ask_number)
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':

        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            
            if data.TimeStamp[i] == x1[temp_ask]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error1 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error2 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error3 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True' 
                
            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                print order_book_ask[temp_ask]['Price'][0:1].iloc[0]
                print data_ask[data_ask_number:data_ask_number+1]['BestPrice'].iloc[0]
                
                if data_ask_number == 0:
                    print 'Best Price is True'
                    pass
                elif data_ask_number > 0 and order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data_ask[data_ask_number-1:data_ask_number]['BestPrice'].iloc[0]:
                    print 'Best Price is True'
                    pass
                else:
                    print 'Best Price is False'
                    pass
                #print temp_ask
                
                #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data[(data.BidOrAsk == 'A')][i:i]['BestPrice'].iloc[0]:
                #    print 'Best Price is True'
                #    continue
                #else:
                #    print 'Best Price is False'
                #   break
                    
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    #print order_book_ask[temp_ask]
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_ask[temp_ask]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print 'position = %d'%(position_)
                    #print 'length-1 = %d'%(len(order_book_ask[temp_ask])-1)
                    #print order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_ask[temp_ask]['Price'][0:1].iloc[0]# != data['BestPrice'][i]
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            #print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
                    #print order_book_ask[temp_ask]                        
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1

                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1

                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)

                        
                    if position_ == 0:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:
                    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            print 'Some error222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        continue                            
                            
                            
                            
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x1[temp_ask]: 
                #print temp_ask
                #print order_book_ask[temp_ask]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]

                if position_ == 0 and len(order_book_ask[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error1111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error2222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error3333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x1[temp_ask]:
                ###a.append(x1[temp_ask])
                #print temp_ask
                #print order_book_ask[temp_ask]
                print order_book_ask[temp_ask]['Price'][0:1].iloc[0]
                print data_ask[data_ask_number:data_ask_number+1]['BestPrice'].iloc[0]
                
                if data_ask_number == 0:
                    print 'Best Price is True'
                    pass
                elif data_ask_number > 0 and order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data_ask[data_ask_number-1:data_ask_number]['BestPrice'].iloc[0]:
                    print 'Best Price is True'
                    pass
                else:
                    print 'Best Price is False'
                    pass
                #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                #    print 'Best Price is True'
                #    continue
                #else:
                #    print 'Best Price is False'
                #    break
                    
                order_book_ask.append(order_book_ask[temp_ask])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                #print 'temp_ask2 = %d'%(temp_ask)
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_ask[temp_ask]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error11111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error22222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_ask[temp_ask])-1:
                        
                    if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error33333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue                        
                    else:
                        print 'position and Best Price is True'                        
                        continue 
                elif position_ == 0 and len(order_book_ask[0]) == 1:
                    print 'position and Best Price is True'
                    continue                          

    elif data.BidOrAsk[i] == 'B':
        #print i
        #print int(data[['QuantityDifference']][i:i+1].values) > 0
        data_bid_number = data_bid_number + 1
        print data_bid_number
        
        if int(data[['QuantityDifference']][i:i+1].values) > 0: #and len(order_book_bid[0]) > 0:
            print x2[temp_bid]
            print data.TimeStamp[i] == x2[temp_bid]
            
            if data.TimeStamp[i] == x2[temp_bid]:
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                #print position_
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error222222 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'   
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:
                        
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error333333 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue
                
            elif data.TimeStamp[i] != x2[temp_bid]:
                
                print 'order book best price = %d'%(order_book_bid[temp_bid]['Price'][0:1].iloc[0])
                print ''
                print 'last data best price = %d'%(data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0])
                print data_bid_number
                
                if data_bid_number == 0:
                    print 'Best Price is True'
                    pass
                elif data_bid_number > 0 and order_book_bid[temp_bid]['Price'][0:1].iloc[0] == data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0]:
                    print 'Best Price is True'
                    pass
                else:
                    print 'Best Price is False'
                    pass
                #data_bid_number = data_bid_number + 1   
                ###b.append(x2[temp_bid])
                #print 'temp bid = %d'%(temp_bid)
                #print order_book_bid[temp_bid]
                    
                if temp_bid == 0:
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    #print order_book_bid[temp_bid]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)                  
                    #print order_book_bid[temp_bid]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    #print position_
                    #print len(order_book_bid[temp_bid]) - 1
                    
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]
                    #print order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0]
                    #print order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0]
                    #print temp_bid
                    #print order_book_bid[temp_bid]
                    #print data['BestPrice'][i]
                    #print order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0]
                    if position_ == 0  and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1111111 %d %d %d'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                            
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error2222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error3333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            continue
                        else:
                            print 'position and Best Price is True'
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            continue
                else:
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference']][i:i+1],int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    #print position_    
                    #print len(order_book_bid[temp_bid])-1      
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error11111111 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                     
                            continue
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:
                    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error22222222 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'  
                            continue
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error33333333 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                            break
                        else:
                            print 'position and Best Price is True'                        
                            continue
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        continue
                        
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    

            if data.TimeStamp[i] == x2[temp_bid]: 
                #print temp_bid
                #print order_book_bid[temp_bid]
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]

                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error19 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
                        
                elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error29 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                        print 'Some error39 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue

            elif data.TimeStamp[i] != x2[temp_bid]:

                print 'order book best price = %d'%(order_book_bid[temp_bid]['Price'][0:1].iloc[0])
                print ''
                print 'last data best price = %d'%(data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0])
                print data_bid_number
                
                if data_bid_number == 0:
                    print 'Best Price is True'
                    pass
                elif data_bid_number > 0 and order_book_bid[temp_bid]['Price'][0:1].iloc[0] == data_bid[data_bid_number-2:data_bid_number-1]['BestPrice'].iloc[0]:
                    print 'Best Price is True'
                    pass
                else:
                    print 'Best Price is False'
                    pass
                
                #if order_book_ask[temp_ask]['Price'][0:1].iloc[0] == data['BestPrice'][i-1]:
                #    print 'Best Price is True'
                #    continue
                #else:
                #    print 'Best Price is False'
                #    break                
                ###b.append(x2[temp_bid])
                #print temp_bid
                #print order_book_bid[temp_bid]
                order_book_bid.append(order_book_bid[temp_bid])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                #print position_
                #print len(order_book_ask[temp_ask])-1
                #print 'temp_bid2 = %d'%(temp_bid)
                
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                #print order_book_bid[temp_bid]
                
                if position_ == 0 and len(order_book_bid[0]) > 1:
                    if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error110 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'                     
                        continue
    
                elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                    if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                        print 'Some error220 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    else:
                        print 'position and Best Price is True'  
                        continue
                elif position_ == len(order_book_bid[temp_bid])-1:
                        
                    if position_ > 0 and order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                        print 'Some error330 %d %d'%(i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                        break
                    elif position_ == 0:
                        print 'position and Best Price is True'
                        continue
                    else:
                        print 'position and Best Price is True'                        
                        continue    
                elif position_ == 0 and len(order_book_bid[0]) == 1:
                    print 'position and Best Price is True'
                    continue                    
                                    
end = time.time()            
print "Total time = %f"%(end - start)    

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:26: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


------------------------------------------------------------------------------
12
                        TimeStamp
12  2014-01-24D04:21:49.422665000
     Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
12  673500  61CF08BD2E520E12                   4        B     674500
data_ask_number = 10
4
2014-01-24D04:21:49.419665000
False
order book best price = 674000

last data best price = 674000
4
Best Price is True
position and Best Price is True
------------------------------------------------------------------------------
13
                        TimeStamp
13  2014-01-24D04:21:49.422665000
     Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
13  673500  6D92ACE99A8D7EAD                   6        B     674500
data_ask_number = 11
5
2014-01-24D04:21:49.422665000
True
position and Best Price is True
------------------------------------------------------------------------------
14
                        TimeStamp
14  2014-01-24D04:21:49.422665000
     Price

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:520: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
199
                         TimeStamp
199  2014-01-24D08:57:27.940246300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
199  641500  4B960CB5ACB48FD4                  -4        B     675000
data_ask_number = 197
94
order book best price = 675000

last data best price = 675000
94
Best Price is True
position and Best Price is True
------------------------------------------------------------------------------
200
                         TimeStamp
200  2014-01-24D08:57:27.940246300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
200  641500  4B960CB5ACB48FD4                   5        B     675000
data_ask_number = 198
95
2014-01-24D08:57:27.940246300
True
position and Best Price is True
------------------------------------------------------------------------------
201
                         TimeStamp
201  2014-01-24D08:57:28.376246

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                         TimeStamp
395  2014-01-24D09:00:24.113971300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
395  675500  61C72CB9AFFC8EFC                   5        A     675000
data_ask_number = 393
position and Best Price is True
------------------------------------------------------------------------------
396
                         TimeStamp
396  2014-01-24D09:00:27.031932300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
396  674500  E918A0B52CF20E32                  -3        B     674000
data_ask_number = 394
239
order book best price = 674500

last data best price = 674500
239
Best Price is True
position and Best Price is True
------------------------------------------------------------------------------
397
                         TimeStamp
397  2014-01-24D09:00:27.031932300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
397  674500  E4AFEB50D3E8B608                 -20        B     674000
data_as

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
411
                         TimeStamp
411  2014-01-24D09:00:27.039932300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
411  675000  4D9F88ED6D3050D0                   5        A     675000
data_ask_number = 409
position and Best Price is True
------------------------------------------------------------------------------
412
                         TimeStamp
412  2014-01-24D09:00:27.044932300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
412  675000  4D9F88ED6D3050D0                  -5        A     675000
data_ask_number = 410
675000
674000
Best Price is False
position and Best Price is True
------------------------------------------------------------------------------
413
                         TimeStamp
413  2014-01-24D09:00:27.044932300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
413  675500  6198

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


position and Best Price is True
------------------------------------------------------------------------------
417
                         TimeStamp
417  2014-01-24D09:00:29.068906300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
417  672000  EDDE08F597EC7B0C                  -5        B     674000
data_ask_number = 415
250
order book best price = 674000

last data best price = 674000
250
Best Price is True
position and Best Price is True
------------------------------------------------------------------------------
418
                         TimeStamp
418  2014-01-24D09:00:29.068906300
      Price       OrderNumber  QuantityDifference BidOrAsk  BestPrice
418  672500  ED18ACEDC343A683                  -5        B     674000
data_ask_number = 416
251
position and Best Price is True
------------------------------------------------------------------------------
419
                         TimeStamp
419  2014-01-24D09:00:34.745828300
      Price       OrderNumbe

In [65]:
len(order_book_ask)

1157

In [67]:
len(data[0:3200][(data[0:3200].BidOrAsk == 'A')].TimeStamp.unique())

1157

In [68]:
len(order_book_bid)

1121

In [70]:
len(data[0:3200][(data[0:3200].BidOrAsk == 'B')].TimeStamp.unique())

1121

In [70]:
a = data[(data.BidOrAsk == 'A')].index.tolist()[9]  
#data[(data.BidOrAsk == 'A')].head(20) #['BestPrice'].iloc[0]
data[(data.BidOrAsk == 'A')][16:17]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
25,CNF14,26,2014-01-24D08:45:00.161957700,66326B58E3EDC66D,13,707000,20,,A,677000,19


In [78]:
data[(data.BidOrAsk == 'A')].reindex()[0:1]#.head()   #.tolist()

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity
0,CNF14,1,2014-01-24D04:21:49.419665000,C791DA0F799A5E9A,1,710000,2,,A,677000,19


In [89]:
data_ask = data[(data.BidOrAsk == 'A')].reindex()

j = 0

for i in range(0,20,1):
    print '------------------------------------------------------------------------------'
    print i
    print data[i:i+1]
    #print data_ask[j:j+1]
    #print data.BidOrAsk[i] == 'A'
    if data.BidOrAsk[i] == 'A':
        print data_ask[j:j+1]['BestPrice'].iloc[0]
        j = j + 1 
    else:
        print data_ask[j-1:j]
        
        
        
        
        

------------------------------------------------------------------------------
0
  Series  SequenceNumber                      TimeStamp       OrderNumber  \
0  CNF14               1  2014-01-24D04:21:49.419665000  C791DA0F799A5E9A   

   OrderBookPosition   Price  QuantityDifference Trade BidOrAsk  BestPrice  \
0                  1  710000                   2              A     677000   

   BestQuantity  
0            19  
677000
------------------------------------------------------------------------------
1
  Series  SequenceNumber                      TimeStamp       OrderNumber  \
1  CNF14               2  2014-01-24D04:21:49.419665000  E1C9F25394A679A6   

   OrderBookPosition   Price  QuantityDifference Trade BidOrAsk  BestPrice  \
1                  2  719500                   1              A     677000   

   BestQuantity  
1            19  
677000
------------------------------------------------------------------------------
2
  Series  SequenceNumber                      T